In [1]:
from preprocessing import Preprocess
from go import GameState, BLACK, WHITE, EMPTY
import os, sgf
import numpy as np

# input SGF files
FILE_FOLDER = os.path.join(os.path.expanduser('~'), 'python', 'tutorial_files','test')
# output archive SGF files
SUCCEED_FOLDER = os.path.join(os.path.expanduser('~'), 'python', 'tutorial_files','succeed')
FAIL_FOLDER = os.path.join(os.path.expanduser('~'), 'python', 'tutorial_files','fail')
# output database
DATA_FOLDER = os.path.join(os.path.expanduser('~'), 'python', 'tutorial_data','go')
TRAIN_DATA = os.path.join(DATA_FOLDER,'train_data')
TEST_DATA = os.path.join(DATA_FOLDER,'test_data')

# BOARD_POSITION contains SGF symbol which represents each row (or column) of the board
# It can be used to convert between 0,1,2,3... and a,b,c,d...
# Symbol [tt] or [] represents PASS in SGF, therefore is omitted
BOARD_POSITION = 'abcdefghijklmnopqrs'

In [2]:
from caffe2.python import core, utils
from caffe2.proto import caffe2_pb2

def write_db(db_type, db_name, base_name, features, labels):
    db = core.C.create_db(db_type, db_name, core.C.Mode.write)
    transaction = db.new_transaction()
    for i in range(features.shape[0]):
        feature_and_label = caffe2_pb2.TensorProtos()
        feature_and_label.protos.extend([
             utils.NumpyArrayToCaffe2Tensor(features[i]),
             utils.NumpyArrayToCaffe2Tensor(labels[i])
        ])
        transaction.put(
            '{}_{:0=3}'.format(base_name,i),
            feature_and_label.SerializeToString())
    # Close the transaction, and then close the db.
    del transaction
    del db

In [3]:
#%%capture output
p = Preprocess()
for dirname, subDirList, fileList in os.walk(FILE_FOLDER):
    for filename in fileList:
        with open(os.path.join(dirname, filename)) as f:
            collection = sgf.parse(f.read())
            for game in collection:
                if game.nodes[0].properties['SZ'] == ['19']: # Size of the Board should only be 19x19
                    state = GameState() # Initialize GameState
                    features = np.empty(shape=(0,48,19,19), dtype=np.int8)
                    labels = np.empty(shape=(0,1), dtype=np.uint16)
                    try:
                        for node in game.nodes[1:]: # Except nodes[0] for game properties
                            features = np.append(features, p.state_to_tensor(state).astype(np.int8), axis=0)
                            if 'B' in node.properties and len(node.properties['B'][0]) == 2: # Black move
                                x = BOARD_POSITION.index(node.properties['B'][0][0])
                                y = BOARD_POSITION.index(node.properties['B'][0][1])
                                state.do_move(action=(x,y),color = BLACK)
                            elif 'W' in node.properties and len(node.properties['W'][0]) == 2: # White move
                                x = BOARD_POSITION.index(node.properties['W'][0][0])
                                y = BOARD_POSITION.index(node.properties['W'][0][1])
                                state.do_move(action=(x,y),color = WHITE)
                            labels = np.append(labels, np.asarray([[x * 19 + y]], dtype=np.uint16), axis=0)
                        write_db(
                            db_type = 'leveldb',
                            db_name = TEST_DATA, # replace this with TRAIN_DATA or TEST_DATA if you want to separate the dataset
                            base_name = os.path.basename(filename),
                            features = features,
                            labels = labels
                        )
                        os.rename(f.name,os.path.join(SUCCEED_FOLDER,filename)) # move the file to SUCCEED_FOLDER, so Preprocess can resume after interrupted
                        print('{} succeeded'.format(filename))
                    except Exception as e:
                        os.rename(f.name,os.path.join(FAIL_FOLDER,filename)) # move the file to FAIL_FOLDER, so Preprocess can resume after interrupted
                        print('{} failed dues to {}'.format(filename, e))

2017-05-11-1.sgf succeeded
2017-05-01-1.sgf succeeded
2017-05-01-10.sgf succeeded
2017-05-01-11.sgf succeeded
2017-05-01-12.sgf succeeded
2017-05-01-14.sgf succeeded
2017-05-01-15.sgf succeeded
2017-05-01-2.sgf succeeded
2017-05-01-3.sgf succeeded
2017-05-01-4.sgf succeeded
2017-05-01-5.sgf succeeded
2017-05-01-6.sgf succeeded
2017-05-01-7.sgf succeeded
2017-05-01-8.sgf succeeded
2017-05-01-9.sgf succeeded
2017-05-02-1.sgf succeeded
2017-05-02-10.sgf succeeded
2017-05-02-12.sgf succeeded
2017-05-02-13.sgf succeeded
2017-05-02-14.sgf succeeded
2017-05-02-15.sgf succeeded
2017-05-02-16.sgf succeeded
2017-05-02-17.sgf succeeded
2017-05-02-18.sgf succeeded
2017-05-02-19.sgf succeeded
2017-05-02-2.sgf succeeded
2017-05-02-21.sgf succeeded
2017-05-02-22.sgf succeeded
2017-05-02-23.sgf succeeded
2017-05-02-24.sgf succeeded
2017-05-02-25.sgf succeeded
2017-05-02-26.sgf succeeded
2017-05-02-3.sgf succeeded
2017-05-02-5.sgf succeeded
2017-05-02-6.sgf succeeded
2017-05-02-7.sgf succeeded
2017-05-

2017-05-12-4.sgf succeeded
2017-05-12-5.sgf succeeded
2017-05-12-6.sgf succeeded
2017-05-12-7.sgf succeeded
2017-05-12-8.sgf succeeded
2017-05-13-1.sgf succeeded
2017-05-13-10.sgf succeeded
2017-05-13-12.sgf succeeded
2017-05-13-13.sgf succeeded
2017-05-13-14.sgf succeeded
2017-05-13-15.sgf succeeded
2017-05-13-16.sgf succeeded
2017-05-13-17.sgf succeeded
2017-05-13-18.sgf succeeded
2017-05-13-2.sgf succeeded
2017-05-13-20.sgf succeeded
2017-05-13-21.sgf succeeded
2017-05-13-22.sgf succeeded
2017-05-13-23.sgf succeeded
2017-05-13-24.sgf succeeded
2017-05-13-25.sgf succeeded
2017-05-13-26.sgf succeeded
2017-05-13-27.sgf succeeded
2017-05-13-29.sgf succeeded
2017-05-13-3.sgf succeeded
2017-05-13-30.sgf succeeded
2017-05-13-31.sgf succeeded
2017-05-13-32.sgf succeeded
2017-05-13-33.sgf succeeded
2017-05-13-34.sgf succeeded
2017-05-13-35.sgf succeeded
2017-05-13-36.sgf succeeded
2017-05-13-37.sgf succeeded
2017-05-13-38.sgf succeeded
2017-05-13-39.sgf succeeded
2017-05-13-4.sgf succeeded
2

2017-05-22-22.sgf succeeded
2017-05-22-23.sgf succeeded
2017-05-22-24.sgf succeeded
2017-05-22-25.sgf succeeded
2017-05-22-4.sgf succeeded
2017-05-22-5.sgf succeeded
2017-05-22-6.sgf succeeded
2017-05-22-7.sgf succeeded
2017-05-22-8.sgf succeeded
2017-05-23-1.sgf succeeded
2017-05-23-10.sgf succeeded
2017-05-23-11.sgf succeeded
2017-05-23-13.sgf succeeded
2017-05-23-14.sgf succeeded
2017-05-23-15.sgf succeeded
2017-05-23-16.sgf succeeded
2017-05-23-17.sgf succeeded
2017-05-23-18.sgf succeeded
2017-05-23-19.sgf succeeded
2017-05-23-2.sgf succeeded
2017-05-23-20.sgf succeeded
2017-05-23-21.sgf succeeded
2017-05-23-22.sgf succeeded
2017-05-23-4.sgf succeeded
2017-05-23-5.sgf succeeded
2017-05-23-6.sgf succeeded
2017-05-23-7.sgf succeeded
2017-05-23-8.sgf succeeded
2017-05-24-1.sgf succeeded
2017-05-24-11.sgf succeeded
2017-05-24-12.sgf succeeded
2017-05-24-13.sgf succeeded
2017-05-24-14.sgf succeeded
2017-05-24-15.sgf succeeded
2017-05-24-16.sgf succeeded
2017-05-24-17.sgf succeeded
2017-

In [9]:
from caffe2.python import model_helper, workspace
model = model_helper.ModelHelper(name="test", arg_scope={"order": "NCHW"})
data, label = model.TensorProtosDBInput(
    [],
    ["data", "label"],
    batch_size=10,
    db=TEST_DATA, # replace this with TRAIN_DATA or TEST_DATA accordingly
    db_type='leveldb')
workspace.RunNetOnce(model.param_init_net)
workspace.CreateNet(model.net, overwrite=True)
workspace.RunNetOnce(model.net)
workspace.FetchBlob("data")[1,2]

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1,